# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from sec import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "YUM"
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Yum Brands Inc.',
 'marketcap': 35777175861,
 'week52high': 132.24,
 'week52low': 103.49,
 'week52highSplitAdjustOnly': 132.87,
 'week52lowSplitAdjustOnly': 103.97,
 'week52change': 0.03337876044993182,
 'sharesOutstanding': 281687866,
 'float': 0,
 'avg10Volume': 1577638,
 'avg30Volume': 1419182,
 'day200MovingAvg': 119.91,
 'day50MovingAvg': 128.73,
 'employees': 2020,
 'ttmEPS': 4.4,
 'ttmDividendRate': 2.263877104969058,
 'dividendYield': 0.017824400480033524,
 'nextDividendDate': '',
 'exDividendDate': '2022-11-23',
 'nextEarningsDate': '2023-02-08',
 'peRatio': 27.86384412823988,
 'beta': 0.6896769291288999,
 'maxChangePercent': 6.872195363827941,
 'year5ChangePercent': 0.6726323580152289,
 'year2ChangePercent': 0.2780005997073911,
 'year1ChangePercent': 0.03337876044993182,
 'ytdChangePercent': -0.0083541536539663,
 'month6ChangePercent': 0.0683029199382621,
 'month3ChangePercent': 0.04772636531919328,
 'month1ChangePercent': -0.012977929748212658,
 'day30Change

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data["year1ChangePercent"]

0.03337876044993182

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        try: 
             append_dataframe = pd.DataFrame([[symbol, 
                                               data[symbol]['quote']['latestPrice'], 
                                               data[symbol]['stats']['year1ChangePercent'],
                                               'N/A']], 
                                      columns= my_columns)
             final_dataframe = pd.concat([final_dataframe,append_dataframe], ignore_index=True)
        except KeyError:
            print(f"stock{symbol} is not on the list")   
    

stockDISCA is not on the list
stockHFC is not on the list
stockVIAC is not on the list
stockWLTW is not on the list


In [13]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,154.55,0.102373,N/A
1,AAL,17.02,0.03465,N/A
2,AAP,152.17,-0.305482,N/A
3,AAPL,154.50,-0.098586,N/A
4,ABBV,145.20,0.071647,N/A
...,...,...,...,...
496,YUM,127.01,0.033379,N/A
497,ZBH,129.45,0.062685,N/A
498,ZBRA,330.81,-0.342444,N/A
499,ZION,54.41,-0.21444,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [14]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop=True, inplace= True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,N/A
1,FTI,13.14,0.935199,N/A
2,HRB,39.33,0.694134,N/A
3,NUE,176.63,0.613898,N/A
4,UNM,41.55,0.585593,N/A
5,LW,98.12,0.583158,N/A
6,VAR,177.07,0.571162,N/A
7,CAH,78.67,0.562606,N/A
8,VLO,131.73,0.556025,N/A
9,MPC,118.20,0.551055,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [25]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
portfolio_input()
print(portfolio_size)

1000000


In [26]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0,len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, "Number of Shares to Buy"] =  math.floor(position_size/final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,245
1,FTI,13.14,0.935199,1492
2,HRB,39.33,0.694134,498
3,NUE,176.63,0.613898,111
4,UNM,41.55,0.585593,471
5,LW,98.12,0.583158,199
6,VAR,177.07,0.571162,110
7,CAH,78.67,0.562606,249
8,VLO,131.73,0.556025,148
9,MPC,118.20,0.551055,165


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [81]:
hqm_columns = [
              'Ticker',
              'Price',
              'Number of Shares to Buy',
              'One-Year Price Return',
              'One-Year Return Percentile',
              'Six-Month Price Return',
              'Six-Month Return Percentile',
              'Three-Month Price Return',
              'Three-Month Return Percentile',
              'One-Month Price Return',
              'One-Month Return Percentile',
              'HQM Score'
              ]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
convert_none = lambda x : 0 if x is None else x

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        try:
            append_dataframe = pd.DataFrame([[symbol, 
                                              convert_none(data[symbol]['quote']['latestPrice']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['year1ChangePercent']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['month6ChangePercent']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['month3ChangePercent']),
                                              'N/A',
                                              convert_none(data[symbol]['stats']['month1ChangePercent']),
                                              'N/A',
                                              'N/A']], columns=hqm_columns)
            hqm_dataframe= pd.concat([hqm_dataframe,append_dataframe], ignore_index=True)
        except KeyError:
            print(f'Stock {symbol} isnt in the list anymore')

Stock DISCA isnt in the list anymore
Stock HFC isnt in the list anymore
Stock VIAC isnt in the list anymore
Stock WLTW isnt in the list anymore


In [82]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,154.55,N/A,0.102373,N/A,0.159067,N/A,0.137422,N/A,0.018989,N/A,N/A
1,AAL,17.02,N/A,0.034650,N/A,0.145357,N/A,0.206237,N/A,0.252391,N/A,N/A
2,AAP,152.17,N/A,-0.305482,N/A,-0.190089,N/A,-0.150727,N/A,0.001843,N/A,N/A
3,AAPL,154.50,N/A,-0.098586,N/A,-0.064189,N/A,0.117901,N/A,0.222697,N/A,N/A
4,ABBV,145.20,N/A,0.071647,N/A,0.065878,N/A,0.008950,N/A,-0.104525,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.01,N/A,0.033379,N/A,0.068303,N/A,0.047726,N/A,-0.012978,N/A,N/A
497,ZBH,129.45,N/A,0.062685,N/A,0.146764,N/A,0.236963,N/A,0.007707,N/A,N/A
498,ZBRA,330.81,N/A,-0.342444,N/A,0.000696,N/A,0.434811,N/A,0.257881,N/A,N/A
499,ZION,54.41,N/A,-0.214440,N/A,0.036154,N/A,0.102361,N/A,0.083217,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [83]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.708583
1      0.590818
2      0.053892
3      0.303393
4      0.658683
         ...   
496    0.588822
497    0.646707
498    0.035928
499    0.129741
500    0.225549
Name: One-Year Return Percentile, Length: 501, dtype: object
0      0.746507
1      0.720559
2      0.035928
3      0.215569
4      0.540918
         ...   
496    0.548902
497    0.722555
498    0.371257
499    0.469062
500    0.275449
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      0.610778
1      0.766467
2      0.015968
3      0.550898
4      0.241517
         ...   
496    0.357285
497    0.814371
498    0.974052
499    0.508982
500    0.848303
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      0.367265
1      0.958084
2      0.291417
3      0.944112
4      0.017964
         ...   
496    0.209581
497    0.327345
498    0.962076
499     0.61477
500    0.772455
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,154.55,N/A,0.102373,0.708583,0.159067,0.746507,0.137422,0.610778,0.018989,0.367265,N/A
1,AAL,17.02,N/A,0.034650,0.590818,0.145357,0.720559,0.206237,0.766467,0.252391,0.958084,N/A
2,AAP,152.17,N/A,-0.305482,0.053892,-0.190089,0.035928,-0.150727,0.015968,0.001843,0.291417,N/A
3,AAPL,154.50,N/A,-0.098586,0.303393,-0.064189,0.215569,0.117901,0.550898,0.222697,0.944112,N/A
4,ABBV,145.20,N/A,0.071647,0.658683,0.065878,0.540918,0.008950,0.241517,-0.104525,0.017964,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.01,N/A,0.033379,0.588822,0.068303,0.548902,0.047726,0.357285,-0.012978,0.209581,N/A
497,ZBH,129.45,N/A,0.062685,0.646707,0.146764,0.722555,0.236963,0.814371,0.007707,0.327345,N/A
498,ZBRA,330.81,N/A,-0.342444,0.035928,0.000696,0.371257,0.434811,0.974052,0.257881,0.962076,N/A
499,ZION,54.41,N/A,-0.214440,0.129741,0.036154,0.469062,0.102361,0.508982,0.083217,0.61477,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: